<a href="https://colab.research.google.com/github/Shivani-Pathak/ANN--Basics/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install evaluate

In [2]:
# Clear dataset cache
# You can manually remove the cache directory or use the datasets library's function if available.
# A simple way for testing is to remove the cache directory.
# Find the cache directory by running:
# from datasets import utils.info_utils; print(utils.info_utils.get_cache_directory())
# For now, we'll proceed assuming a clean state or that updating handles potential issues.

!pip install --upgrade datasets transformers evaluate huggingface_hub fsspec torch

In [3]:

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import BertTokenizerFast, BertForSequenceClassification
from datasets import load_dataset
import evaluate
import matplotlib.pyplot as plt

# 0. Config
model_checkpoint = 'bert-base-uncased'
batch_size = 16
epochs = 1
lr = 2e-5

# 1. Load & subsample IMDb

raw = load_dataset('imdb')
raw_train = raw['train'].shuffle(seed=42).select(range(2500))
train_val = raw_train.train_test_split(test_size=500, seed=42)
train_ds = train_val['train']   # 2000
val_ds   = train_val['test']    # 500
test_ds  = raw['test'].shuffle(seed=42).select(range(500))

# 2. Tokenizer + tokenize_fn
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
def tokenize_fn(batch):
    return tokenizer(
        batch['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

# 3. Preprocess splits
def preprocess(split_ds):
    ds = split_ds.map(tokenize_fn, batched=True)
    ds = ds.rename_column('label', 'labels')
    ds = ds.remove_columns(['text'])
    ds.set_format(type='torch', columns=['input_ids','attention_mask','labels'])
    return ds

train_ds = preprocess(train_ds)
val_ds   = preprocess(val_ds)
test_ds  = preprocess(test_ds)

# 4. DataLoaders
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size)
test_loader  = DataLoader(test_ds,  batch_size=batch_size)

# 5. Model & optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2
).to(device)
optimizer = AdamW(model.parameters(), lr=lr)

# 6. Training
train_losses, val_losses, val_accuracies = [], [], []

for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = batch['input_ids'].to(device)
        masks  = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        out = model(inputs, attention_mask=masks, labels=labels)
        loss = out.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    val_metric = evaluate.load('accuracy')
    model.eval()
    total_vloss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['input_ids'].to(device)
            masks  = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            out = model(inputs, attention_mask=masks, labels=labels)
            total_vloss += out.loss.item()
            preds = out.logits.argmax(dim=-1)
            val_metric.add_batch(predictions=preds.cpu(), references=labels.cpu())
    avg_val_loss = total_vloss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_acc = val_metric.compute()['accuracy']
    val_accuracies.append(val_acc)

    print(
        f"Epoch {epoch}/{epochs}  "
        f"Train loss: {avg_train_loss:.4f}  "
        f"Val loss: {avg_val_loss:.4f}  "
        f"Val acc: {val_acc:.4f}"
    )

# 7. Test evaluation
test_metric = evaluate.load('accuracy')
model.eval()
with torch.no_grad():
    for batch in test_loader:
        inputs = batch['input_ids'].to(device)
        masks  = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        out = model(inputs, attention_mask=masks)
        preds = out.logits.argmax(dim=-1)
        test_metric.add_batch(predictions=preds.cpu(), references=labels.cpu())
test_acc = test_metric.compute()['accuracy']
print(f"\nTest Accuracy: {test_acc:.4f}")

# 8. Visualization
plt.figure(figsize=(6,4))
plt.plot([1], train_losses,    marker='o', label='Train Loss')
plt.plot([1], val_losses,      marker='o', label='Val Loss')
plt.plot([1], val_accuracies,  marker='o', label='Val Acc')
plt.title('Training Metrics (1 epoch, small subset)')
plt.xlabel('Epoch')
plt.xticks([1])
plt.legend()
plt.tight_layout()
plt.show()


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:
# Clear any existing installations to ensure a clean slate
!pip uninstall -y transformers datasets evaluate huggingface_hub fsspec torch torchvision

# Install compatible versions - specifying versions is safer, but letting pip resolve is also an option
# Here, we will install torch and torchvision first, then the others
# You might need to check the PyTorch website for recommended versions based on your CUDA/CPU setup: https://pytorch.org/get-started/locally/
# For CPU only:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# If you have a compatible CUDA GPU (e.g., CUDA 11.8):
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


# Then install the rest of the libraries, ensuring they are compatible with the installed torch/torchvision
!pip install datasets transformers evaluate huggingface_hub fsspec

# Continue with your original code from here:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import BertTokenizerFast, BertForSequenceClassification
from datasets import load_dataset
import evaluate
import matplotlib.pyplot as plt

# 0. Config
model_checkpoint = 'bert-base-uncased'
batch_size = 16
epochs = 1
lr = 2e-5

# 1. Load & subsample IMDb

raw = load_dataset('imdb')
raw_train = raw['train'].shuffle(seed=42).select(range(2500))
train_val = raw_train.train_test_split(test_size=500, seed=42)
train_ds = train_val['train']   # 2000
val_ds   = train_val['test']    # 500
test_ds  = raw['test'].shuffle(seed=42).select(range(500))

# 2. Tokenizer + tokenize_fn
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
def tokenize_fn(batch):
    return tokenizer(
        batch['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

# 3. Preprocess splits
def preprocess(split_ds):
    ds = split_ds.map(tokenize_fn, batched=True)
    ds = ds.rename_column('label', 'labels')
    ds = ds.remove_columns(['text'])
    ds.set_format(type='torch', columns=['input_ids','attention_mask','labels'])
    return ds

train_ds = preprocess(train_ds)
val_ds   = preprocess(val_ds)
test_ds  = preprocess(test_ds)

# 4. DataLoaders
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size)
test_loader  = DataLoader(test_ds,  batch_size=batch_size)

# 5. Model & optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2
).to(device)
optimizer = AdamW(model.parameters(), lr=lr)

# 6. Training
train_losses, val_losses, val_accuracies = [], [], []

for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = batch['input_ids'].to(device)
        masks  = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        out = model(inputs, attention_mask=masks, labels=labels)
        loss = out.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    val_metric = evaluate.load('accuracy')
    model.eval()
    total_vloss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['input_ids'].to(device)
            masks  = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            out = model(inputs, attention_mask=masks, labels=labels)
            total_vloss += out.loss.item()
            preds = out.logits.argmax(dim=-1)
            val_metric.add_batch(predictions=preds.cpu(), references=labels.cpu())
    avg_val_loss = total_vloss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_acc = val_metric.compute()['accuracy']
    val_accuracies.append(val_acc)

    print(
        f"Epoch {epoch}/{epochs}  "
        f"Train loss: {avg_train_loss:.4f}  "
        f"Val loss: {avg_val_loss:.4f}  "
        f"Val acc: {val_acc:.4f}"
    )

# 7. Test evaluation
test_metric = evaluate.load('accuracy')
model.eval()
with torch.no_grad():
    for batch in test_loader:
        inputs = batch['input_ids'].to(device)
        masks  = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        out = model(inputs, attention_mask=masks)
        preds = out.logits.argmax(dim=-1)
        test_metric.add_batch(predictions=preds.cpu(), references=labels.cpu())
test_acc = test_metric.compute()['accuracy']
print(f"\nTest Accuracy: {test_acc:.4f}")

# 8. Visualization
plt.figure(figsize=(6,4))
plt.plot([1], train_losses,    marker='o', label='Train Loss')
plt.plot([1], val_losses,      marker='o', label='Val Loss')
plt.plot([1], val_accuracies,  marker='o', label='Val Acc')
plt.title('Training Metrics (1 epoch, small subset)')
plt.xlabel('Epoch')
plt.xticks([1])
plt.legend()
plt.tight_layout()
plt.show()

Found existing installation: fsspec 2024.6.1
Uninstalling fsspec-2024.6.1:
  Successfully uninstalled fsspec-2024.6.1
Found existing installation: torch 2.7.0+cpu
Uninstalling torch-2.7.0+cpu:
  Successfully uninstalled torch-2.7.0+cpu
Found existing installation: torchvision 0.22.0+cpu
Uninstalling torchvision-0.22.0+cpu:
  Successfully uninstalled torchvision-0.22.0+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.7.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.22.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.7.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl (176.0 MB)
Using cached https://download.pytorch.org/whl/cpu/torchvision-0.22.0%2Bcpu-cp311-cp311-manylinu

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
